<a href="https://colab.research.google.com/github/tomkart/llm_examples/blob/main/ollama_local_tools_weather_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_community langchain-ollama

In [ ]:
# prompt: connect to ollama llm on 192.168.68.72

from langchain_ollama import OllamaLLM
ollama = OllamaLLM(model="llama3.2", base_url="http://192.168.68.72:11434")

In [ ]:
# Example usage
prompt = "What is the meaning of life?"
response = ollama(prompt)
response

<ipython-input-3-6e00db783962>:3: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = ollama(prompt)


'The question of the meaning of life is one of the most profound and debated topics in human history. It\'s a question that has puzzled philosophers, theologians, scientists, and ordinary people for centuries.\n\nThere is no straightforward answer to this question, as it can vary greatly depending on cultural, personal, and philosophical perspectives. However, here are some possible approaches to understanding the meaning of life:\n\n1. **Religious or spiritual perspectives**: Many religions offer answers to the meaning of life, such as seeking happiness through following a deity\'s will, achieving enlightenment, or finding salvation.\n2. **Philosophical perspectives**: Philosophers like Aristotle, Epicurus, and Jean-Paul Sartre have offered various interpretations, including:\n * Pursuing happiness and fulfillment\n * Seeking knowledge and wisdom\n * Living virtuously and cultivating relationships\n * Embracing existentialism and taking responsibility for one\'s choices\n3. **Scientif

In [ ]:
from langchain.agents import tool

In [ ]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f'The current temperature is {current_temperature}°C'

In [ ]:
get_current_temperature.invoke({"latitude": 13, "longitude": 14})

'The current temperature is 15.4°C'

In [ ]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [ ]:
search_wikipedia.invoke({"query": "langchain"})

"Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval-Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  \nUse cases include providing chatbot access to internal company data or giving factual information only from an authoritative source.\n\nPage: Mil

In [ ]:
from langchain_ollama import ChatOllama

tools = [
             search_wikipedia,
             get_current_temperature
            ]


llm = ChatOllama(model="llama3.2",
                    temperature=0,
                    base_url="http://192.168.68.72:11434").bind_tools(tools)

In [ ]:
llm.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-17T04:01:50.7296546Z', 'done': True, 'done_reason': 'stop', 'total_duration': 179596000, 'load_duration': 23558900, 'prompt_eval_count': 236, 'prompt_eval_duration': 9000000, 'eval_count': 30, 'eval_duration': 145000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-99e6cb08-9778-4ad0-8f92-7e0df97bc0a1-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122.4194}, 'id': '202ee5b2-1ac9-423a-a5ba-b724ea67038d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 236, 'output_tokens': 30, 'total_tokens': 266})

In [ ]:
llm.invoke("what is langchain")

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-17T04:01:50.8416173Z', 'done': True, 'done_reason': 'stop', 'total_duration': 105531600, 'load_duration': 12242300, 'prompt_eval_count': 232, 'prompt_eval_duration': 7000000, 'eval_count': 19, 'eval_duration': 85000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-6adebef0-9bf3-4551-a487-6a8f3b7c94e5-0', tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'langchain'}, 'id': '3a4b2c62-823a-42b6-90f8-6efcdbffeeb9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 232, 'output_tokens': 19, 'total_tokens': 251})

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | llm

In [ ]:
result = chain.invoke({"input": "what is the weather in hong kong and sydney"})
result

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-17T04:27:44.8309878Z', 'done': True, 'done_reason': 'stop', 'total_duration': 323427200, 'load_duration': 22134000, 'prompt_eval_count': 247, 'prompt_eval_duration': 11000000, 'eval_count': 60, 'eval_duration': 288000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-38055428-7286-42d7-836e-7e8e3e3ab314-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 22.3964, 'longitude': -114.1095}, 'id': 'f77f60b1-1a3d-4c89-be68-5525fc1ee07a', 'type': 'tool_call'}, {'name': 'get_current_temperature', 'args': {'latitude': -33.8651, 'longitude': 151.2093}, 'id': '91d08643-7f14-4c30-9e00-dca80983321a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 247, 'output_tokens': 60, 'total_tokens': 307})

In [ ]:
type(result)

langchain_core.messages.ai.AIMessage

In [ ]:
result.tool_calls

[{'name': 'get_current_temperature',
  'args': {'latitude': 22.3964, 'longitude': -114.1095},
  'id': 'f77f60b1-1a3d-4c89-be68-5525fc1ee07a',
  'type': 'tool_call'},
 {'name': 'get_current_temperature',
  'args': {'latitude': -33.8651, 'longitude': 151.2093},
  'id': '91d08643-7f14-4c30-9e00-dca80983321a',
  'type': 'tool_call'}]

In [ ]:
tool_call = result.tool_calls[0]  # Get the first (and likely only) tool call
tool_name = tool_call['name']  # Access the tool name using dictionary key
tool_args = tool_call['args']  # Access the tool arguments using dictionary key


In [ ]:
tool_name


'get_current_temperature'

In [ ]:
tool_args

{'latitude': 37.7749, 'longitude': -122.4194}

In [ ]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }
        tool_call = result.tool_calls[0]
        tool_name = tool_call['name']  # Access the tool name using dictionary key
        tool_args = tool_call['args']  # Access the tool arguments using dictionary key

        if tool_name in tools:
            return tools[tool_name].run(tool_args)  # Use run for tools, __call__ for lambdas
        else:
            # Handle unsupported tool names
            return tool.run("langchain")  # Or other appropriate response

In [ ]:
chain = prompt | llm | route

In [ ]:
result = chain.invoke({"input": "what is the weather in sydney now"})
result

'The current temperature is 18.3°C'

In [ ]:
result = chain.invoke({"input": "what is langchain"})
result

"Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval-Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  \nUse cases include providing chatbot access to internal company data or giving factual information only from an authoritative source.\n\nPage: Mil

In [ ]:
result = chain.invoke({"input": "what is your name"})
result

'Page: Perplexity AI\nSummary: Perplexity AI is a conversational search engine that uses large language models (LLMs) to answer queries using sources from the web and cites links within the text response. Its developer, Perplexity AI, Inc., is based in San Francisco, California.\n\nPage: Mistral AI\nSummary: Mistral AI SAS is a French artificial intelligence (AI) startup, headquartered in Paris. It specializes in open-weight large language models (LLMs). Founded in April 2023 by engineers formerly employed by Google DeepMind and Meta Platforms, the company has gained prominence as an alternative to proprietary AI systems. Named after the mistral – a powerful, cold wind in southern France – the company emphasized openness and innovation in the AI field. Mistral AI positions itself as an alternative to proprietary models.\nIn October 2023, Mistral AI raised €385 million. By December 2023, it was valued at over $2 billion.\nIn June 2024, Mistral AI secured a €600 million ($645 million) fu

In [ ]:
result = chain.invoke({"input": "what is weather in sydney? and also weather in hong kong?"})
result

'The current temperature is 18.1°C'

In [ ]:
chain = prompt | llm

result = chain.invoke({"input": "what is weather in sydney"})
result

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-17T04:27:17.8824661Z', 'done': True, 'done_reason': 'stop', 'total_duration': 184023000, 'load_duration': 23134300, 'prompt_eval_count': 241, 'prompt_eval_duration': 7000000, 'eval_count': 30, 'eval_duration': 150000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-8d18dac3-e3b2-4f0a-b1de-07ac29a8a4fe-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 33.865141, 'longitude': -151.209903}, 'id': '9762d1de-9feb-44ca-853c-50a7adbd025c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 241, 'output_tokens': 30, 'total_tokens': 271})